In [ ]:
# İş Problemi

In [ ]:
# İngiltere merkezli perakende şirketi satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir.
# Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.

In [ ]:
# Veri Seti Hikayesi

In [ ]:
# Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış işlemlerini içeriyor.
# Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi mevcuttur.

In [ ]:
# Değişenler

In [ ]:
# InvoiceNo : Fatura Numarası (Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder)
# StockCode : Ürün Kodu (Her bir ürün için eşsiz)
# Description : Ürün İsmi
# Quantity : Ürün Adedi (Faturalardaki ürünlerden kaçar tane satıldığı)
# InvoiceDate : Fatura Tarihi
# UnitPrice : Fatura Fiyatı (Sterlin)
# CustomerID : Eşsiz Müşteri Numarası
# Country : Ülke İsmi

In [ ]:
###################################################
# Gerekli Kütüphaneler ve Fonksiyonlar
###################################################

In [ ]:
! pip install lifetimes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 11.0 MB/s eta 0:00:00


In [ ]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [ ]:
pd.set_option("display.max_columns", None)  # Tüm sütunları göster
pd.set_option("display.width",500)    # Sütunları aşağıdan inmeden göster
pd.set_option("display.float_format", lambda x: "%.4f" % x)   # Virgülden sonra 4 basamağa kadar göster
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Aykırı değerleri bulan fonksiyon
def outlier_thresholds(dataframe, variable):
  quartile1 = dataframe[variable].quantile(0.01)
  quartile3 = dataframe[variable].quantile(0.99)
  interquantile_range = quartile3 - quartile1
  up_limit = quartile3 + 1.5 * interquantile_range
  low_limit = quartile1 - 1.5 * interquantile_range
  return low_limit, up_limit

In [ ]:
# Aykırı değerleri low_limit ve up_limit'e eşitleyen fonksiyon
def replace_with_thresholds(dataframe, variable):
  low_limit, up_limit = outlier_thresholds(dataframe, variable)
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
  dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
###############################################
# Verinin Okunması
###############################################

In [ ]:
df_ = pd.read_excel("/content/online_retail_II.xlsx")

In [ ]:
df = df_.copy()

In [ ]:
df.describe().T      # Describe sadece sayısal değişkenleri bize verecek

,count,mean,min,25%,50%,75%,max,std
Quantity,525461.0000,10.3377,-9600.0000,1.0000,3.0000,10.0000,19152.0000,107.4241
InvoiceDate,525461,2010-06-28 11:37:36.845017856,2009-12-01 07:45:00,2010-03-21 12:20:00,2010-07-06 09:51:00,2010-10-15 12:45:00,2010-12-09 20:01:00,NaN
Price,525461.0000,4.6888,-53594.3600,1.2500,2.1000,4.2100,25111.0900,146.1269
Customer ID,417534.0000,15360.6455,12346.0000,13983.0000,15311.0000,16799.0000,18287.0000,1680.8113


In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.9500,13085.0000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.1000,13085.0000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.2500,13085.0000,United Kingdom


In [ ]:
df.isnull().sum()  # ID 'sini bilmediğim bir müşteriyi analiz edemem o yüzden uçuracağız bu kısımları
# Description ksımında eksik verilerde veri setimize oranla çok az o yüzden bunu da uçuracağız

,0
Invoice,0
StockCode,0
Description,2928
Quantity,0
InvoiceDate,0
Price,0
Customer ID,107927
Country,0


In [ ]:
##############################################################
# Veri Ön İşleme
##############################################################

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,417534.0000,12.7588,-9360.0000,2.0000,4.0000,12.0000,19152.0000,101.2204
InvoiceDate,417534,2010-07-01 01:12:19.787609856,2009-12-01 07:45:00,2010-03-26 11:26:00,2010-07-08 19:12:00,2010-10-14 14:08:00,2010-12-09 20:01:00,NaN
Price,417534.0000,3.8875,0.0000,1.2500,1.9500,3.7500,25111.0900,71.1318
Customer ID,417534.0000,15360.6455,12346.0000,13983.0000,15311.0000,16799.0000,18287.0000,1680.8113


In [ ]:
# İnvoice değişkeninin başında yer alan C ifadesi iadeleri temsil etmektedir.
# İade olan işlemlerde de Quantity ve Price değişkenlerinde eksi değerler almasına neden olmaktadır.
# Bu yüzden bunları veri setinden çıkarmamız gerekiyor.
df = df[~df["Invoice"].str.contains("C", na=False)]

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,407695.0000,13.5867,1.0000,2.0000,5.0000,12.0000,19152.0000,96.8422
InvoiceDate,407695,2010-07-01 10:10:10.782177792,2009-12-01 07:45:00,2010-03-26 14:01:00,2010-07-09 15:46:00,2010-10-14 17:09:00,2010-12-09 20:01:00,NaN
Price,407695.0000,3.2942,0.0000,1.2500,1.9500,3.7500,10953.5000,34.7567
Customer ID,407695.0000,15368.5041,12346.0000,13997.0000,15321.0000,16812.0000,18287.0000,1679.7957


In [ ]:
# Üstte Quantity ve Price değişkenine ait sorunlar çözülmüş gibi ama yine de sıfırdan büyük olanları alsın bakalım
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,407664.0000,13.5856,1.0000,2.0000,5.0000,12.0000,19152.0000,96.8407
InvoiceDate,407664,2010-07-01 10:15:11.871688192,2009-12-01 07:45:00,2010-03-26 14:01:00,2010-07-09 15:47:00,2010-10-14 17:09:00,2010-12-09 20:01:00,NaN
Price,407664.0000,3.2944,0.0010,1.2500,1.9500,3.7500,10953.5000,34.7580
Customer ID,407664.0000,15368.5926,12346.0000,13997.0000,15321.0000,16812.0000,18287.0000,1679.7621


In [ ]:
# Aykırı değerleri baskılama işleme
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,407664.0000,11.9065,1.0000,2.0000,5.0000,12.0000,358.5000,28.7251
InvoiceDate,407664,2010-07-01 10:15:11.871688192,2009-12-01 07:45:00,2010-03-26 14:01:00,2010-07-09 15:47:00,2010-10-14 17:09:00,2010-12-09 20:01:00,NaN
Price,407664.0000,3.0056,0.0010,1.2500,1.9500,3.7500,36.9400,3.2506
Customer ID,407664.0000,15368.5926,12346.0000,13997.0000,15321.0000,16812.0000,18287.0000,1679.7621


In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0000,2009-12-01 07:45:00,6.9500,13085.0000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12.0000,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12.0000,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0000,2009-12-01 07:45:00,2.1000,13085.0000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0000,2009-12-01 07:45:00,1.2500,13085.0000,United Kingdom


In [ ]:
# Faturadaki her bir ürüne toplam ne kadar bedel ödendiğini belirleyelim
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [ ]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0000,2009-12-01 07:45:00,6.9500,13085.0000,United Kingdom,83.4000
1,489434,79323P,PINK CHERRY LIGHTS,12.0000,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom,81.0000
2,489434,79323W,WHITE CHERRY LIGHTS,12.0000,2009-12-01 07:45:00,6.7500,13085.0000,United Kingdom,81.0000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0000,2009-12-01 07:45:00,2.1000,13085.0000,United Kingdom,100.8000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0000,2009-12-01 07:45:00,1.2500,13085.0000,United Kingdom,30.0000
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2.0000,2010-12-09 20:01:00,2.9500,17530.0000,United Kingdom,5.9000
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1.0000,2010-12-09 20:01:00,3.7500,17530.0000,United Kingdom,3.7500
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1.0000,2010-12-09 20:01:00,3.7500,17530.0000,United Kingdom,3.7500
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2.0000,2010-12-09 20:01:00,3.7500,17530.0000,United Kingdom,7.5000


In [ ]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [ ]:
today_date = dt.datetime(2010, 12, 11)  # Analiz yaptığımız gün

In [ ]:
type(today_date)

datetime.datetime

In [ ]:
##########################################################################
# Lifetime Veri Yapısının Hazırlanması
##########################################################################

In [ ]:
# recency : Son satın alma üzerinden geçen zaman. Haftalık. (kullanıcı özelinde).
# Bu recency değeri analiz tarihine göre değil müşterinin kendi içinde son satın alması ile ilk satın alması arasındaki farkı ifade etmektedir.
# T : Müşterinin yaşı. Haftalık. (analiz tarihinden ne kadar süre önce ilk satın alma yapılmış)
# frequency : Tekrar eden toplam satın alma sayısı (frequency>1)
# monetary : Satın alma başına ortalama kazanç

In [ ]:
cltv_df = df.groupby("Customer ID").agg(
    {"InvoiceDate": [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,  # Gün cinsinde recency değerini bize verdi
                     lambda InvoiceDate: (today_date - InvoiceDate.min()).days],    # Müşterinin yaşını hesaplıyor
     "Invoice": lambda Invoice: Invoice.nunique(),     # Eşsiz fatura sayısını hesaplamış olacağız. Frequency değerini bize verecek
     "TotalPrice": lambda TotalPrice: TotalPrice.sum()}     # Satın alma başına ortalama kazanç
)

In [ ]:
cltv_df

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0000          196        361       11   372.8600
12347.0000           37         40        2  1323.3200
12348.0000            0         74        1   222.1600
12349.0000          181        225        3  2295.0200
12351.0000            0         11        1   300.9300
...                 ...        ...      ...        ...
18283.0000          275        294        6   641.7700
18284.0000            0         67        1   448.6200
18285.0000            0        296        1   413.9400
18286.0000          247        359        2  1283.3700
18287.0000          188        207        4  2332.6500

[4312 rows x 4 columns]

In [ ]:
# Üstteki şekilde okunabilirliği düşük. Bunun için üstteki sütun isimlerine silme işlemi yapalım.
cltv_df.columns = cltv_df.columns.droplevel(0)

In [ ]:
cltv_df

,<lambda_0>,<lambda_1>,<lambda>,<lambda>
Customer ID,,,,
12346.0000,196,361,11,372.8600
12347.0000,37,40,2,1323.3200
12348.0000,0,74,1,222.1600
12349.0000,181,225,3,2295.0200
12351.0000,0,11,1,300.9300
...,...,...,...,...
18283.0000,275,294,6,641.7700
18284.0000,0,67,1,448.6200
18285.0000,0,296,1,413.9400


In [ ]:
cltv_df.columns = ["recency", "T", "frequency", "monetary"]

In [ ]:
cltv_df

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,196,361,11,372.8600
12347.0000,37,40,2,1323.3200
12348.0000,0,74,1,222.1600
12349.0000,181,225,3,2295.0200
12351.0000,0,11,1,300.9300
...,...,...,...,...
18283.0000,275,294,6,641.7700
18284.0000,0,67,1,448.6200
18285.0000,0,296,1,413.9400


In [ ]:
# monetary : Satın alma başına ortalama kazanç.
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

In [ ]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.0000,133.9986,132.8272,0.0000,0.0000,105.0000,254.0000,373.0000
T,4312.0000,225.4970,118.9098,1.0000,117.0000,253.0000,329.0000,374.0000
frequency,4312.0000,4.4557,8.1702,1.0000,1.0000,2.0000,5.0000,205.0000
monetary,4312.0000,358.6432,347.7206,2.9500,178.4787,283.1710,418.8906,5633.4650


In [ ]:
# Üstteki tabloda frequency'nin min değeri 1 fakat bizim bulmak istediğimiz frequency : Tekrar eden toplam satın alma sayısı
# Bu yüzden 1'den büyük değerleri seçiyoruz.
cltv_df = cltv_df[(cltv_df["frequency"] > 1)]

In [ ]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,2893.0000,199.7242,114.7534,0.0000,103.0000,209.0000,301.0000,373.0000
T,2893.0000,261.7515,103.3122,7.0000,196.0000,288.0000,360.0000,374.0000
frequency,2893.0000,6.1507,9.5274,2.0000,2.0000,4.0000,7.0000,205.0000
monetary,2893.0000,374.1855,340.0801,10.8222,199.7825,298.9629,436.9967,5633.4650


In [ ]:
cltv_df["recency"] = cltv_df["recency"] / 7  # Bizden recency değeri haftalık olarak isteniyor
cltv_df["T"] = cltv_df["T"] / 7     # Bizden müşteri yaşı haftalık isteniyor

In [ ]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,2893.0000,28.5320,16.3933,0.0000,14.7143,29.8571,43.0000,53.2857
T,2893.0000,37.3931,14.7589,1.0000,28.0000,41.1429,51.4286,53.4286
frequency,2893.0000,6.1507,9.5274,2.0000,2.0000,4.0000,7.0000,205.0000
monetary,2893.0000,374.1855,340.0801,10.8222,199.7825,298.9629,436.9967,5633.4650


In [ ]:
##############################################
# BG-NBD Modelinin Kurulması
##############################################

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)   # Bu parametre bu modelin parametrelerinin bulunması aşamasında katsayılara uygulanacak olan ceza katsayısıdır.

In [ ]:
bgf.fit(cltv_df["frequency"],
        cltv_df["recency"],
        cltv_df["T"])

<lifetimes.BetaGeoFitter: fitted with 2893 subjects, a: 0.14, alpha: 10.85, b: 2.72, r: 2.13>

In [ ]:
###############################################################################
# 1 hafta içinde en çok satın alma beklediğimiz 10 müşteri kimdir?
###############################################################################

In [ ]:
bgf.conditional_expected_number_of_purchases_up_to_time(1,    # Burada 1 haftalık tahmin olduğu için 1 yazdık
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"]).sort_values(ascending=False).head(10)

,0
Customer ID,
14911.0000,3.2155
17850.0000,2.4259
12748.0000,2.2820
15311.0000,1.9107
13089.0000,1.7232
14606.0000,1.6210
14156.0000,1.6040
13694.0000,1.4845
17841.0000,1.4478


In [ ]:
# 1. müşteri 3.21 adet satın alma yapabilir diye beklentideyiz
# 2. müşteri 2.42 adet satın alma yapabilir diye beklentideyiz

In [ ]:
# Şimdi bütün müşteriler için 1 hafta içinde beklediğimiz satın almaları hesaplayıp bunu cltv_df veri setimize ekleyebiliriz
cltv_df["expected_purc_1_week"] = bgf.conditional_expected_number_of_purchases_up_to_time(1,
                                                                                          cltv_df["frequency"],
                                                                                          cltv_df["recency"],
                                                                                          cltv_df["T"])

In [ ]:
cltv_df

,recency,T,frequency,monetary,expected_purc_1_week
Customer ID,,,,,
12346.0000,28.0000,51.5714,11,33.8964,0.0315
12347.0000,5.2857,5.7143,2,661.6600,0.2383
12349.0000,25.8571,32.1429,3,765.0067,0.1116
12352.0000,2.2857,4.0000,2,171.9000,0.2604
12356.0000,6.2857,8.5714,3,1187.4167,0.2490
...,...,...,...,...,...
18276.0000,48.0000,52.1429,5,264.1320,0.1093
18277.0000,13.8571,18.5714,4,256.6650,0.1939
18283.0000,39.2857,42.0000,6,106.9617,0.1494


In [ ]:
##################################################################
# 1 ay içinde en çok satın alma beklediğimiz 10 müşteri kimdir?
##################################################################

In [ ]:
bgf.conditional_expected_number_of_purchases_up_to_time(4,
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"]).sort_values(ascending=False).head(10)

,0
Customer ID,
14911.0000,12.8204
17850.0000,9.6720
12748.0000,9.0980
15311.0000,7.6179
13089.0000,6.8705
14606.0000,6.4630
14156.0000,6.3953
13694.0000,5.9187
17841.0000,5.7725


In [ ]:
# Şimdi bütün müşteriler için 1 ay içinde beklediğimiz satın almaları hesaplayıp bunu cltv_df veri setimize ekleyebiliriz
cltv_df["expected_purc_1_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4,
                                                                                           cltv_df["frequency"],
                                                                                           cltv_df["recency"],
                                                                                           cltv_df["T"])

In [ ]:
cltv_df

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month
Customer ID,,,,,,
12346.0000,28.0000,51.5714,11,33.8964,0.0315,0.1255
12347.0000,5.2857,5.7143,2,661.6600,0.2383,0.9414
12349.0000,25.8571,32.1429,3,765.0067,0.1116,0.4442
12352.0000,2.2857,4.0000,2,171.9000,0.2604,1.0276
12356.0000,6.2857,8.5714,3,1187.4167,0.2490,0.9856
...,...,...,...,...,...,...
18276.0000,48.0000,52.1429,5,264.1320,0.1093,0.4359
18277.0000,13.8571,18.5714,4,256.6650,0.1939,0.7699
18283.0000,39.2857,42.0000,6,106.9617,0.1494,0.5954


In [ ]:
# Şirketimizde 1 ay içerisinde ne kadar satın alma olabileceğini görelim
bgf.conditional_expected_number_of_purchases_up_to_time(4,
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"]).sum()

np.float64(1792.9810117736124)

In [ ]:
# 3 ayda tüm şirketin beklenen satış sayısı nedir?

In [ ]:
bgf.conditional_expected_number_of_purchases_up_to_time(4 * 3,
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"]).sum()

np.float64(5309.198520344429)

In [ ]:
cltv_df["expected_purc_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 3,
                                                                                           cltv_df["frequency"],
                                                                                           cltv_df["recency"],
                                                                                           cltv_df["T"])

In [ ]:
cltv_df

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month
Customer ID,,,,,,,
12346.0000,28.0000,51.5714,11,33.8964,0.0315,0.1255,0.3733
12347.0000,5.2857,5.7143,2,661.6600,0.2383,0.9414,2.7480
12349.0000,25.8571,32.1429,3,765.0067,0.1116,0.4442,1.3163
12352.0000,2.2857,4.0000,2,171.9000,0.2604,1.0276,2.9927
12356.0000,6.2857,8.5714,3,1187.4167,0.2490,0.9856,2.8869
...,...,...,...,...,...,...,...
18276.0000,48.0000,52.1429,5,264.1320,0.1093,0.4359,1.2964
18277.0000,13.8571,18.5714,4,256.6650,0.1939,0.7699,2.2709
18283.0000,39.2857,42.0000,6,106.9617,0.1494,0.5954,1.7682


In [ ]:
# 2. GAMMA - GAMMA Modelinin Kurulması

In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.01)    # Bu parametre bu modelin parametrelerinin bulunması aşamasında katsayılara uygulanacak olan ceza katsayısıdır.

In [ ]:
ggf.fit(cltv_df["frequency"], cltv_df["monetary"])

<lifetimes.GammaGammaFitter: fitted with 2893 subjects, p: 3.79, q: 0.34, v: 3.73>

In [ ]:
ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                        cltv_df["monetary"]).head(10)

,0
Customer ID,
12346.0000,34.7855
12347.0000,726.7538
12349.0000,813.4300
12352.0000,190.3223
12356.0000,1261.8495
12357.0000,6172.3459
12358.0000,958.0482
12359.0000,436.1604
12360.0000,325.9717


In [ ]:
# Azalan bir şekilde ilk 10 gözlemi görelim
ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                        cltv_df["monetary"]).sort_values(ascending=False).head(10)

,0
Customer ID,
12357.0000,6172.3459
17450.0000,5457.7883
14091.0000,5022.9216
14088.0000,4792.1915
18102.0000,3533.3926
12409.0000,3228.0113
14646.0000,3115.7658
12454.0000,2940.2847
16684.0000,2866.3781


In [ ]:
# Şimdi Conditional Expected Average Profit (Koşul Sağlandığında Beklenen Ortalama Kar) için sütun oluşturalım
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                             cltv_df["monetary"])

In [ ]:
# Şimdi sıralayım expected_average_profit'e göre
cltv_df.sort_values("expected_average_profit", ascending=False).head(10)

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit
Customer ID,,,,,,,,
12357.0000,0.0000,3.4286,2,5633.4650,0.2573,1.0150,2.9534,6172.3459
17450.0000,10.0000,10.5714,7,5321.5850,0.4162,1.6493,4.8410,5457.7883
14091.0000,6.2857,47.7143,2,4584.0450,0.0099,0.0393,0.1167,5022.9216
14088.0000,2.0000,8.1429,3,4512.9900,0.2199,0.8702,2.5478,4792.1915
18102.0000,53.2857,53.4286,89,3526.4424,1.4136,5.6359,16.7696,3533.3926
12409.0000,20.5714,28.7143,4,3086.6550,0.1404,0.5584,1.6535,3228.0113
14646.0000,51.7143,53.2857,78,3108.7672,1.2320,4.9120,14.6155,3115.7658
12454.0000,3.5714,8.1429,4,2811.4450,0.2835,1.1219,3.2849,2940.2847
16684.0000,50.2857,52.5714,27,2847.7670,0.4523,1.8033,5.3648,2866.3781


In [ ]:
##########################################################################
# BG-NBD ve Gamma Gamma Modeli ile CLTV'nin Hesaplanaması
##########################################################################

In [ ]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency"],
                                   cltv_df["T"],
                                   cltv_df["monetary"],
                                   time=3,         # 3 Haftalık
                                   freq="W",         # T'nin frekans bilgisi (Girmiş olduğun veri aylık mı haftalık mı günlük mü olduğunu belirler)
                                   discount_rate=0.01)      # Zaman içinde yaptığımız ürünlerde indirim yapabilirsin bunu da göz önünde bulunduruyoruz

In [ ]:
cltv.head()

,clv
Customer ID,
12346.0000,13.8130
12347.0000,2120.5530
12349.0000,1138.6045
12352.0000,604.6512
12356.0000,3869.2110


In [ ]:
cltv = cltv.reset_index()
cltv.head()

,Customer ID,clv
0,12346.0000,13.8130
1,12347.0000,2120.5530
2,12349.0000,1138.6045
3,12352.0000,604.6512
4,12356.0000,3869.2110


In [ ]:
# Şimdi cltv_df ile cltv'yi birleştirelim
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv
2827,18102.0000,53.2857,53.4286,89,3526.4424,1.4136,5.6359,16.7696,3533.3926,63037.4868
1096,14646.0000,51.7143,53.2857,78,3108.7672,1.2320,4.9120,14.6155,3115.7658,48446.5167
846,14156.0000,52.2857,53.4286,102,1791.7474,1.6040,6.3953,19.0293,1794.8458,36335.9152
1229,14911.0000,53.2857,53.4286,205,698.9298,3.2155,12.8204,38.1474,699.5418,28389.9651
2499,17450.0000,10.0000,10.5714,7,5321.5850,0.4162,1.6493,4.8410,5457.7883,28068.6218
616,13694.0000,51.5714,53.0000,94,1364.9177,1.4845,5.9187,17.6104,1367.4888,25619.7519
5,12357.0000,0.0000,3.4286,2,5633.4650,0.2573,1.0150,2.9534,6172.3459,19350.4200
2523,17511.0000,52.7143,53.2857,31,2726.2206,0.5130,2.0455,6.0860,2741.7306,17751.5867
2099,16684.0000,50.2857,52.5714,27,2847.7670,0.4523,1.8033,5.3648,2866.3781,16359.4486
1300,15061.0000,53.0000,53.4286,86,938.1211,1.3657,5.4449,16.2012,940.0665,16202.8775


In [ ]:
######################################################
# CLTV'ye Göre Segmentlerin Oluşturulması
######################################################

In [ ]:
cltv_final

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv
0,12346.0000,28.0000,51.5714,11,33.8964,0.0315,0.1255,0.3733,34.7855,13.8130
1,12347.0000,5.2857,5.7143,2,661.6600,0.2383,0.9414,2.7480,726.7538,2120.5530
2,12349.0000,25.8571,32.1429,3,765.0067,0.1116,0.4442,1.3163,813.4300,1138.6045
3,12352.0000,2.2857,4.0000,2,171.9000,0.2604,1.0276,2.9927,190.3223,604.6512
4,12356.0000,6.2857,8.5714,3,1187.4167,0.2490,0.9856,2.8869,1261.8495,3869.2110
...,...,...,...,...,...,...,...,...,...,...
2888,18276.0000,48.0000,52.1429,5,264.1320,0.1093,0.4359,1.2964,274.4279,378.4616
2889,18277.0000,13.8571,18.5714,4,256.6650,0.1939,0.7699,2.2709,269.3124,650.0210
2890,18283.0000,39.2857,42.0000,6,106.9617,0.1494,0.5954,1.7682,110.7961,208.3789
2891,18286.0000,35.2857,51.2857,2,641.6850,0.0587,0.2341,0.6961,704.8753,521.9814


In [ ]:
cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])
# 0 le %25 arası değerlere D
# %25 ile %50 arası değerlere C
# %50 ile %75 arası değerlere B
# %75 ile %100 arası değerlere A der.

In [ ]:
# Şimdi bu değerleri büyükten küçüğe sıralayım
cltv_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv,segment
2827,18102.0000,53.2857,53.4286,89,3526.4424,1.4136,5.6359,16.7696,3533.3926,63037.4868,A
1096,14646.0000,51.7143,53.2857,78,3108.7672,1.2320,4.9120,14.6155,3115.7658,48446.5167,A
846,14156.0000,52.2857,53.4286,102,1791.7474,1.6040,6.3953,19.0293,1794.8458,36335.9152,A
1229,14911.0000,53.2857,53.4286,205,698.9298,3.2155,12.8204,38.1474,699.5418,28389.9651,A
2499,17450.0000,10.0000,10.5714,7,5321.5850,0.4162,1.6493,4.8410,5457.7883,28068.6218,A
616,13694.0000,51.5714,53.0000,94,1364.9177,1.4845,5.9187,17.6104,1367.4888,25619.7519,A
5,12357.0000,0.0000,3.4286,2,5633.4650,0.2573,1.0150,2.9534,6172.3459,19350.4200,A
2523,17511.0000,52.7143,53.2857,31,2726.2206,0.5130,2.0455,6.0860,2741.7306,17751.5867,A
2099,16684.0000,50.2857,52.5714,27,2847.7670,0.4523,1.8033,5.3648,2866.3781,16359.4486,A
1300,15061.0000,53.0000,53.4286,86,938.1211,1.3657,5.4449,16.2012,940.0665,16202.8775,A


In [ ]:
# Bütün değerleri daha iyi görebilmek için segment'e göre groupby'a alalım
cltv_final.groupby("segment").agg(
    {"count", "mean", "sum"}
)

Customer ID                     recency                        T                  frequency             monetary                   expected_purc_1_week                expected_purc_1_month                expected_purc_3_month                 expected_average_profit                         clv                   
               mean           sum count    mean        sum count    mean        sum count      mean   sum count     mean         sum count                 mean      sum count                  mean      sum count                  mean       sum count                    mean         sum count      mean          sum count
segment                                                                                                                                                                                                                                                                                                                         
D        15527.8522 11242165.0000   724 21.9943 15923.8571   724 41.0529 29722.2857   724    3.2486  2352   724 199.9623 144772.7395   724               0.0653  47.2967   724                0.2601 188.2843   724                0.7711  558.3026   724                215.9256 156330.1066   724  140.3912  101643.1973   724
C        15444.6542 11166485.0000   723 30.0692 21740.0000   723 37.5527 27150.5714   723    4.2254  3055   723 267.5635 193448.4314   723               0.1219  88.1122   723                0.4849 350.6055   723                1.4365 1038.6014   723                284.8501 205946.6389   723  381.4726  275804.6830   723
B        15302.5795 11063765.0000   723 30.1237 21779.4286   723 35.4145 25604.7143   723    5.5367  4003   723 373.3217 269911.6088   723               0.1621 117.1893   723                0.6445 465.9974   723                1.9068 1378.5875   723                393.7591 284687.8039   723  677.8842  490110.2909   723
A        15128.0581 10937586.0000   723 31.9500 23099.8571   723 35.5471 25700.5714   723   11.5961  8384   723 656.1355 474385.9572   723               0.2740 198.1079   723                1.0900 788.0938   723                3.2278 2333.7070   723                682.9001 493736.7592   723 2227.3224 1610354.0809   723

In [ ]:
# Case 1 : Veri setini kullanarak UK'deki müşteriler için 6 aylık CLTV tahmini yapınız.

In [ ]:
cltv_uk = cltv_df[cltv_df.index.isin(df[df["Country"] == "United Kingdom"]["Customer ID"])]

cltv_uk = ggf.customer_lifetime_value(bgf,
                                      cltv_df["frequency"],
                                      cltv_df["recency"],
                                      cltv_df["T"],
                                      cltv_df["monetary"],
                                      time= 6,
                                      freq="M",
                                      discount_rate=0.01)

In [ ]:
cltv_uk = cltv_uk.reset_index()
cltv_uk.sort_values(by="clv", ascending=False).head(10)

,Customer ID,clv
2827,18102.0000,28793.3936
1096,14646.0000,22129.1072
846,14156.0000,16596.9793
2499,17450.0000,12967.6643
1229,14911.0000,12967.4405
616,13694.0000,11702.7334
5,12357.0000,9002.8943
2523,17511.0000,8108.7039
2099,16684.0000,7473.3916
1300,15061.0000,7400.9315


In [ ]:
cltv_uk_final = cltv_df.merge(cltv_uk, on="Customer ID", how="left")
cltv_uk_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv
2827,18102.0000,53.2857,53.4286,89,3526.4424,1.4136,5.6359,16.7696,3533.3926,28793.3936
1096,14646.0000,51.7143,53.2857,78,3108.7672,1.2320,4.9120,14.6155,3115.7658,22129.1072
846,14156.0000,52.2857,53.4286,102,1791.7474,1.6040,6.3953,19.0293,1794.8458,16596.9793
2499,17450.0000,10.0000,10.5714,7,5321.5850,0.4162,1.6493,4.8410,5457.7883,12967.6643
1229,14911.0000,53.2857,53.4286,205,698.9298,3.2155,12.8204,38.1474,699.5418,12967.4405
616,13694.0000,51.5714,53.0000,94,1364.9177,1.4845,5.9187,17.6104,1367.4888,11702.7334
5,12357.0000,0.0000,3.4286,2,5633.4650,0.2573,1.0150,2.9534,6172.3459,9002.8943
2523,17511.0000,52.7143,53.2857,31,2726.2206,0.5130,2.0455,6.0860,2741.7306,8108.7039
2099,16684.0000,50.2857,52.5714,27,2847.7670,0.4523,1.8033,5.3648,2866.3781,7473.3916
1300,15061.0000,53.0000,53.4286,86,938.1211,1.3657,5.4449,16.2012,940.0665,7400.9315


In [ ]:
# Yüksek CLTV'ye sahip müşteriler için strateji: Bu tür müşterilerle ilişkilerinizi güçlendirmek, onları elde tutmak bizim için çok önemli.
# Bu yüzden bu müşteriler için sadakat programları, VIP hizmetler ya da özelleştirlmiş teklifler sunulabilir.

In [ ]:
# Orta düzey CLTV'ye sahip müşterler için strateji : Bu tür müşterler için, alışveriş sıklığını ve harcalamaları artırmaya yönelik pazarlama kampanyaları yapılabilir.
# Özellikle sık alışveriş yapmalarını sağlayacak indirimi artırma veya müşteriye özel fırsatlar sunulabilir.

In [ ]:
# Düşük CLTV'ye sahip müşteriler için strateji : Bu müşterileri daha değerli hale getirebilmek için onları sadık müşterilere dönüştürme stratejleri geliştirilmelidir.
# Kampanyalar veya kişisel teklifler ile daha fazla harcama yapmaları teşvik edilebilir.

In [ ]:
# Case 2: Veri setini kullanarak UK müşterileri için 1 aylık ve 12 aylık CLTV hesaplayınız ve 1 aylık CLTV'de en yüksek olan 10 kişi ile 12 aylık'taki en yüksek 10 kişiyi analiz ediniz.

In [ ]:
# 1 ay için CLTV değeri
cltv_uk_one_month = cltv_df[cltv_df.index.isin(df[df["Country"] == "United Kingdom"]["Customer ID"])]

cltv_uk_one_month = ggf.customer_lifetime_value(bgf,
                                      cltv_df["frequency"],
                                      cltv_df["recency"],
                                      cltv_df["T"],
                                      cltv_df["monetary"],
                                      time= 1,
                                      freq="M",
                                      discount_rate=0.01)

In [ ]:
cltv_uk_one_month = cltv_uk_one_month.reset_index()    # Güzel gözükmesi için indexleri resetliyoruz
cltv_uk_one_month.sort_values(by="clv", ascending=False).head(10)    # Büyükten küçüğe sıralama

,Customer ID,clv
2827,18102.0000,4945.1933
1096,14646.0000,3800.6698
846,14156.0000,2850.4845
2499,17450.0000,2249.3101
1229,14911.0000,2227.1070
616,13694.0000,2009.9826
5,12357.0000,1572.6966
2523,17511.0000,1392.7010
2099,16684.0000,1283.6650
1300,15061.0000,1271.0921


In [ ]:
cltv_uk_one_month_final = cltv_df.merge(cltv_uk_one_month, on="Customer ID", how="left")
cltv_uk_one_month_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv
2827,18102.0000,53.2857,53.4286,89,3526.4424,1.4136,5.6359,16.7696,3533.3926,4945.1933
1096,14646.0000,51.7143,53.2857,78,3108.7672,1.2320,4.9120,14.6155,3115.7658,3800.6698
846,14156.0000,52.2857,53.4286,102,1791.7474,1.6040,6.3953,19.0293,1794.8458,2850.4845
2499,17450.0000,10.0000,10.5714,7,5321.5850,0.4162,1.6493,4.8410,5457.7883,2249.3101
1229,14911.0000,53.2857,53.4286,205,698.9298,3.2155,12.8204,38.1474,699.5418,2227.1070
616,13694.0000,51.5714,53.0000,94,1364.9177,1.4845,5.9187,17.6104,1367.4888,2009.9826
5,12357.0000,0.0000,3.4286,2,5633.4650,0.2573,1.0150,2.9534,6172.3459,1572.6966
2523,17511.0000,52.7143,53.2857,31,2726.2206,0.5130,2.0455,6.0860,2741.7306,1392.7010
2099,16684.0000,50.2857,52.5714,27,2847.7670,0.4523,1.8033,5.3648,2866.3781,1283.6650
1300,15061.0000,53.0000,53.4286,86,938.1211,1.3657,5.4449,16.2012,940.0665,1271.0921


In [ ]:
# 12 ay için CLTV değerini hesaplayalım
cltv_uk_twelve_month = cltv_df[cltv_df.index.isin(df[df["Country"] == "United Kingdom"]["Customer ID"])]

cltv_uk_twelve_month = ggf.customer_lifetime_value(bgf,
                                      cltv_df["frequency"],
                                      cltv_df["recency"],
                                      cltv_df["T"],
                                      cltv_df["monetary"],
                                      time= 12,
                                      freq="M",
                                      discount_rate=0.01)

In [ ]:
cltv_uk_twelve_month = cltv_uk_twelve_month.reset_index()     # Güzel gözükmesi için indexleri sıfırlıyoruz
cltv_uk_twelve_month.sort_values(by="clv", ascending=False).head(10)    # Büyükten küçüğe sıralama işlemi

,Customer ID,clv
2827,18102.0000,55588.5754
1096,14646.0000,42721.8870
846,14156.0000,32042.2194
1229,14911.0000,25035.1824
2499,17450.0000,24796.8739
616,13694.0000,22592.5167
5,12357.0000,17113.1404
2523,17511.0000,15654.0668
2099,16684.0000,14426.6138
1300,15061.0000,14288.2437


In [ ]:
cltv_uk_twelve_month_final = cltv_df.merge(cltv_uk_twelve_month, on="Customer ID", how="left")
cltv_uk_twelve_month_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv
2827,18102.0000,53.2857,53.4286,89,3526.4424,1.4136,5.6359,16.7696,3533.3926,55588.5754
1096,14646.0000,51.7143,53.2857,78,3108.7672,1.2320,4.9120,14.6155,3115.7658,42721.8870
846,14156.0000,52.2857,53.4286,102,1791.7474,1.6040,6.3953,19.0293,1794.8458,32042.2194
1229,14911.0000,53.2857,53.4286,205,698.9298,3.2155,12.8204,38.1474,699.5418,25035.1824
2499,17450.0000,10.0000,10.5714,7,5321.5850,0.4162,1.6493,4.8410,5457.7883,24796.8739
616,13694.0000,51.5714,53.0000,94,1364.9177,1.4845,5.9187,17.6104,1367.4888,22592.5167
5,12357.0000,0.0000,3.4286,2,5633.4650,0.2573,1.0150,2.9534,6172.3459,17113.1404
2523,17511.0000,52.7143,53.2857,31,2726.2206,0.5130,2.0455,6.0860,2741.7306,15654.0668
2099,16684.0000,50.2857,52.5714,27,2847.7670,0.4523,1.8033,5.3648,2866.3781,14426.6138
1300,15061.0000,53.0000,53.4286,86,938.1211,1.3657,5.4449,16.2012,940.0665,14288.2437


In [ ]:
# Buradan şunları çıkartabiliriz:
# İlk 10 müşteri sıralaması 1 ve 12 aylık periyotlarda büyük ölçüde aynı kaldı.
# Bu müşteri davranışlarının tutarlı olduğunu ve en yüksek değeri yaratan müşterilerin hem kısa hemde uzun vadede işletmeye yüksek kazanç sağlayabileceğini gösteriyor.
# Özellikle ilk 3 müşteri her iki zaman diliminde de sırasını korumuş. Bu müşterileri sadık ve sürekli alışveriş yapan bireyler olduğu söylenebilir.
# 1 aylık CLTV ile 12 aylık CLTV değerleri arasında çok büyük farklar var.
# Örneğin, Customer ID 18102: 1 aylık CLTV -> 4.945 iken 12 aylık CLTV -> 55.588
# Bu durum modelin zamanla daha fazla tahmin edilen sipariş ve kar beklentisiyle CLTV'yi çarpan etkisiyle büyüttüğünü gösteriyor.
# Ayrıca en yüksek CLTV'ye sahip müşteriler için sadakat programları, özel indiririmler, kampanyalar veya Upsell - Cross- Sell fırsatları sunulabilir.

In [ ]:
# Case 3: Veri Setimizde UK müşterileri için 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.
#  4 grup içerisinden seçeceğiniz 2 grup için yönetime kısa kısa 6 aylık aksiyon önerilerinde bulununuz.

In [ ]:
cltv_uk_final["segment"] = pd.qcut(cltv_uk_final["clv"], 4, labels=["D", "C", "B", "A"])   # qcut fonksiyonu küçükten büyüğe sıralama yapar
# 0 ile %25 arası değerlere D
# %25 ile %50 arası değerlere C
# %50 ile %75 arası değerlere B
# %75 ile %100 arası değerlere A der.

In [ ]:
# Şimdi bu değerleri küçükten büyüğe sıralayalım
cltv_uk_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_purc_3_month,expected_average_profit,clv,segment
2827,18102.0000,53.2857,53.4286,89,3526.4424,1.4136,5.6359,16.7696,3533.3926,28793.3936,A
1096,14646.0000,51.7143,53.2857,78,3108.7672,1.2320,4.9120,14.6155,3115.7658,22129.1072,A
846,14156.0000,52.2857,53.4286,102,1791.7474,1.6040,6.3953,19.0293,1794.8458,16596.9793,A
2499,17450.0000,10.0000,10.5714,7,5321.5850,0.4162,1.6493,4.8410,5457.7883,12967.6643,A
1229,14911.0000,53.2857,53.4286,205,698.9298,3.2155,12.8204,38.1474,699.5418,12967.4405,A
616,13694.0000,51.5714,53.0000,94,1364.9177,1.4845,5.9187,17.6104,1367.4888,11702.7334,A
5,12357.0000,0.0000,3.4286,2,5633.4650,0.2573,1.0150,2.9534,6172.3459,9002.8943,A
2523,17511.0000,52.7143,53.2857,31,2726.2206,0.5130,2.0455,6.0860,2741.7306,8108.7039,A
2099,16684.0000,50.2857,52.5714,27,2847.7670,0.4523,1.8033,5.3648,2866.3781,7473.3916,A
1300,15061.0000,53.0000,53.4286,86,938.1211,1.3657,5.4449,16.2012,940.0665,7400.9315,A


In [ ]:
# Bütün değerleri daha iyi görebilmek için segment'e göre groupby'a alalım
cltv_uk_final.groupby("segment").agg(
    {"count", "mean", "sum"}
)

Customer ID                     recency                        T                  frequency             monetary                   expected_purc_1_week                expected_purc_1_month                expected_purc_3_month                 expected_average_profit                         clv                  
               mean           sum count    mean        sum count    mean        sum count      mean   sum count     mean         sum count                 mean      sum count                  mean      sum count                  mean       sum count                    mean         sum count      mean         sum count
segment                                                                                                                                                                                                                                                                                                                        
D        15527.9392 11242228.0000   724 22.0142 15938.2857   724 41.0848 29745.4286   724    3.2486  2352   724 200.1238 144889.6495   724               0.0653  47.2559   724                0.2598 188.1226   724                0.7705  557.8271   724                216.0970 156454.2152   724   64.2794  46538.2897   724
C        15446.9184 11168122.0000   723 30.1454 21795.1429   723 37.6311 27207.2857   723    4.2310  3059   723 268.0045 193767.2764   723               0.1217  87.9585   723                0.4841 350.0025   723                1.4341 1036.8689   723                285.2921 206266.1937   723  174.8025 126382.2088   723
B        15304.5920 11065220.0000   723 30.1832 21822.4286   723 35.4760 25649.1429   723    5.5615  4021   723 373.2375 269850.7297   723               0.1622 117.2757   723                0.6450 466.3520   723                1.9083 1379.7036   723                393.6035 284575.3291   723  310.9025 224782.5067   723
A        15123.6943 10934431.0000   723 31.7943 22987.2857   723 35.3752 25576.2857   723   11.5657  8362   723 655.6170 474011.0814   723               0.2742 198.2159   723                1.0906 788.5039   723                3.2293 2334.7990   723                682.4420 493405.5706   723 1021.3220 738415.8293   723

In [ ]:
# Segment A (En Değerli Müşteriler) : Bu grup en yüksek harcama, en sık satın alma ve en yüksek CLTV'ye sahip. Toplam gelirin çok büyük bir kısmını oluşturuyor.
# 6 aylık aksiyon önerileri olarak ise:
# 1- Sadakat Programları Başlatılabilir : Bu müşterilere özel VIP sadakat programları sunarak, rakipleri geçmelerini engelleyebiliriz.
# Örneğin ; her 5 alışverişte 1 ücretsiz gönderim, özel puanlar, doğum günü kampanyaları gibi
# 2- Kişiselleştirilmiş Pazarlama : Alışveriş geçmişlerine göre ürün önerileri, kişiye özel e-mail ve SMS kampanyaları yapılabilir. "Sizin için seçtik" gibi içeriklerle bağlılığı güçlendirebiliriz.
# 3- Upsell & Cross-sell Kampanyaları : Bu grup daha yüksek sepet tutarına sahip. Onlara yüksek fiyatlı veya tamamlayıcı ürünler sunulabilir.
# 4- Erken Erişim & Özel İndirimler : Yeni gelen ürünlere ilk erişim hakkı, sınırlı stok ürünlere özel fırsatlar tanımlanabilir.

In [ ]:
# Segment D (En Az Değerli Müşteriler) : Bu grup düşük frekans ve çok düşük CLTV'ye sahip. Ama yine de potansiyl taşıyan, kazanılabilir bri segment olabilir.
# 6 aylık aksiyon önerileri ise:
# 1- İlk Alışveriş Teşviki : Özellikle ücretsiz kargo, ilk alışverişe özel %20 indirim gibi teşviklerle bu segmenti aktifleştirmeye çalışabiliriz.
# 2- Geri Kazanım Kampanyaları : Uzun süredir işlem yapmamış müşterilere özel hatırlatma e-postaları gönderin: "Sizi özledik! Size özel 30TL indirim!"
# 3- Dijital İçeriklerle Etkileşim : Bu müşterilere blog içerikler, kullanım rehberleri, popüler ürünleri listele gibi içeriklerle markayı hatırlayın.
# 4- Segment Bazlı A/B Testi : Hangi kampanya türüne daha iyi yanıt verdiklerini öğrenmek için denemeler yapılabilir.